In [ ]:
# 1. IMPORT REQUIRED LIBRARIES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn preprocessing and models
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

# Visualization
from sklearn.metrics import ConfusionMatrixDisplay

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ All libraries imported successfully!")

# 🔮 Customer Churn Prediction System
## End-to-End Machine Learning Pipeline

This notebook builds a complete churn prediction system to identify customers likely to stop using a service. Using classification models and business-focused analysis, we'll uncover key churn drivers and create actionable insights for retention strategies.

**Dataset**: Telco Customer Churn (7,043 customers)  
**Target**: Binary classification (Churned: Yes/No)  
**Models**: Logistic Regression, Random Forest, XGBoost

## 2. Load and Explore Customer Data

Loading the Telco Customer Churn dataset from our data folder and examining its structure, shape, and key statistics.

In [ ]:
# Load data from CSV
data_path = Path('data/telco_churn_processed.csv')
if not data_path.exists():
    data_path = Path('data/telco_churn.csv')

df = pd.read_csv(data_path)

print(f"📊 Dataset Shape: {df.shape}")
print(f"\nFirst 5 rows:")
print(df.head())
print(f"\nData Types:")
print(df.dtypes)
print(f"\nMissing Values:")
print(df.isnull().sum().sum())

In [ ]:
print(f"\n📈 Summary Statistics:")
print(df.describe())

## 3. Data Cleaning and Preprocessing

Handling missing values, ensuring data quality, and preparing categorical variables for modeling.

In [ ]:
# Check for missing values in detail
print("Checking data quality...")
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    if df['TotalCharges'].isnull().sum() > 0:
        print(f"⚠️  Found {df['TotalCharges'].isnull().sum()} missing values in TotalCharges")
        df['TotalCharges'].fillna(df['MonthlyCharges'], inplace=True)
        print("✓ Filled missing TotalCharges with MonthlyCharges")

# Remove duplicates
initial_shape = df.shape[0]
df = df.drop_duplicates()
print(f"✓ Removed {initial_shape - df.shape[0]} duplicate rows")

# Check categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\n📋 Categorical Columns ({len(categorical_cols)}):")
for col in categorical_cols:
    print(f"  - {col}: {df[col].nunique()} unique values")

## 4. Exploratory Data Analysis (EDA)

Visualizing churn distribution and relationships between features and churn target.

In [ ]:
# Churn distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot
churn_counts = df['Churn'].value_counts()
churn_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'], alpha=0.7)
axes[0].set_title('Churn Distribution', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
for i, v in enumerate(churn_counts.values):
    axes[0].text(i, v + 100, str(v), ha='center', fontweight='bold')

# Pie chart
churn_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'])
axes[1].set_title('Churn Proportion', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

# Calculate churn rate
churn_rate = (df['Churn'] == 'Yes').sum() / len(df) * 100
print(f"\n💡 Churn Rate: {churn_rate:.2f}%")

In [ ]:
# Churn by Contract Type
if 'Contract' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 5))
    contract_churn = df.groupby('Contract')['Churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)
    contract_churn.sort_values(ascending=False).plot(kind='barh', ax=ax, color='#e74c3c', alpha=0.7)
    ax.set_title('Churn Rate by Contract Type', fontsize=12, fontweight='bold')
    ax.set_xlabel('Churn Rate (%)')
    for i, v in enumerate(contract_churn.values):
        ax.text(v + 1, i, f'{v:.1f}%', va='center')
    plt.tight_layout()
    plt.show()

# Churn by Internet Service
if 'InternetService' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 5))
    internet_churn = df.groupby('InternetService')['Churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)
    internet_churn.sort_values(ascending=False).plot(kind='barh', ax=ax, color='#3498db', alpha=0.7)
    ax.set_title('Churn Rate by Internet Service', fontsize=12, fontweight='bold')
    ax.set_xlabel('Churn Rate (%)')
    for i, v in enumerate(internet_churn.values):
        ax.text(v + 1, i, f'{v:.1f}%', va='center')
    plt.tight_layout()
    plt.show()

In [ ]:
# Tenure impact on churn
if 'tenure' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 5))
    for status in ['No', 'Yes']:
        data = df[df['Churn'] == status]['tenure']
        ax.hist(data, alpha=0.6, label=f'Churn: {status}', bins=30)
    ax.set_xlabel('Tenure (months)')
    ax.set_ylabel('Count')
    ax.set_title('Tenure Distribution by Churn Status', fontsize=12, fontweight='bold')
    ax.legend()
    plt.tight_layout()
    plt.show()

# Monthly Charges impact
if 'MonthlyCharges' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 5))
    for status in ['No', 'Yes']:
        data = df[df['Churn'] == status]['MonthlyCharges']
        ax.hist(data, alpha=0.6, label=f'Churn: {status}', bins=30)
    ax.set_xlabel('Monthly Charges ($)')
    ax.set_ylabel('Count')
    ax.set_title('Monthly Charges Distribution by Churn Status', fontsize=12, fontweight='bold')
    ax.legend()
    plt.tight_layout()
    plt.show()

## 5. Feature Engineering

Creating new features to enhance model performance and interpretability.

In [ ]:
# Create a copy for feature engineering
df_features = df.copy()

# Tenure-based features
if 'tenure' in df_features.columns:
    df_features['tenure_group'] = pd.cut(df_features['tenure'], 
                                          bins=[0, 6, 12, 24, 73],
                                          labels=['0-6m', '6-12m', '1-2y', '2+y'])
    print("✓ Created tenure_group feature")

# Charge-based features
if 'MonthlyCharges' in df_features.columns and 'TotalCharges' in df_features.columns:
    df_features['avg_monthly_to_total'] = df_features['MonthlyCharges'] / (df_features['TotalCharges'] + 1)
    df_features['total_charges_group'] = pd.qcut(df_features['TotalCharges'], 
                                                   q=4, 
                                                   labels=['Low', 'Medium', 'High', 'VeryHigh'],
                                                   duplicates='drop')
    print("✓ Created charge ratio and charge group features")

# Service adoption score
service_cols = [col for col in df_features.columns if 'Service' in col or 'Security' in col or 'Support' in col]
if service_cols:
    df_features['service_adoption_score'] = df_features[service_cols].apply(lambda x: (x == 'Yes').sum(), axis=1)
    print(f"✓ Created service adoption score from {len(service_cols)} service columns")

print(f"\n📝 New feature columns: {df_features.columns.tolist()[-5:]}")

## 6. Prepare Data for Modeling

Encoding categorical variables and scaling numerical features.

In [ ]:
# Prepare features and target
X = df_features.copy()

# Drop non-predictive columns
drop_columns = ['customerID', 'Churn', 'tenure_group', 'total_charges_group']
X = X.drop(columns=[col for col in drop_columns if col in X.columns])

# Encode categorical variables
label_encoders = {}
categorical_cols = X.select_dtypes(include=['object']).columns

print(f"Encoding {len(categorical_cols)} categorical columns...")
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    print(f"  ✓ {col}")

# Handle NaN values
X = X.fillna(X.mean(numeric_only=True))

# Target variable
y = df_features['Churn'].map({'No': 0, 'Yes': 1})

print(f"\n✓ Final feature matrix shape: {X.shape}")
print(f"✓ Target variable shape: {y.shape}")
print(f"✓ Class distribution: {y.value_counts().to_dict()}")

In [ ]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ Train set shape: {X_train_scaled.shape}")
print(f"✓ Test set shape: {X_test_scaled.shape}")
print(f"✓ Train churn distribution: {np.bincount(y_train)}")
print(f"✓ Test churn distribution: {np.bincount(y_test)}")

## 7. Build and Train Classification Models

Training three classification models: Logistic Regression, Random Forest, and XGBoost.

In [ ]:
print("🚀 Training Classification Models...\n")

models = {}
predictions = {}

# 1. Logistic Regression
print("1️⃣  Logistic Regression...")
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)
models['Logistic Regression'] = lr
predictions['Logistic Regression'] = {
    'y_pred': lr.predict(X_test_scaled),
    'y_pred_proba': lr.predict_proba(X_test_scaled)[:, 1]
}
print("   ✓ Model trained")

# 2. Random Forest
print("2️⃣  Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15)
rf.fit(X_train_scaled, y_train)
models['Random Forest'] = rf
predictions['Random Forest'] = {
    'y_pred': rf.predict(X_test_scaled),
    'y_pred_proba': rf.predict_proba(X_test_scaled)[:, 1]
}
print("   ✓ Model trained")

# 3. XGBoost
print("3️⃣  XGBoost...")
xgb = XGBClassifier(
    n_estimators=200,
    max_depth=7,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss',
    verbosity=0
)
xgb.fit(X_train_scaled, y_train)
models['XGBoost'] = xgb
predictions['XGBoost'] = {
    'y_pred': xgb.predict(X_test_scaled),
    'y_pred_proba': xgb.predict_proba(X_test_scaled)[:, 1]
}
print("   ✓ Model trained")

print("\n✓ All models trained successfully!")

## 8. Model Evaluation and Comparison

Computing and comparing key metrics for all models.

In [ ]:
print("📊 MODEL EVALUATION RESULTS\n")
print("=" * 80)

results = {}

for model_name in models.keys():
    y_pred = predictions[model_name]['y_pred']
    y_pred_proba = predictions[model_name]['y_pred_proba']
    
    results[model_name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba),
        'Confusion Matrix': confusion_matrix(y_test, y_pred)
    }
    
    print(f"\n{model_name}:")
    print(f"  Accuracy:  {results[model_name]['Accuracy']:.4f}")
    print(f"  Precision: {results[model_name]['Precision']:.4f}")
    print(f"  Recall:    {results[model_name]['Recall']:.4f}")
    print(f"  F1-Score:  {results[model_name]['F1-Score']:.4f}")
    print(f"  ROC-AUC:   {results[model_name]['ROC-AUC']:.4f}")

print("\n" + "=" * 80)

In [ ]:
# Create comparison dataframe
results_df = pd.DataFrame(results).T
print("\n📈 Model Performance Comparison:")
print(results_df[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']].round(4))

In [ ]:
# Visualization: Model Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Metrics comparison
metrics_plot_df = results_df[['Accuracy', 'ROC-AUC', 'Precision', 'Recall', 'F1-Score']]
metrics_plot_df.plot(kind='bar', ax=axes[0], alpha=0.7)
axes[0].set_title('Model Performance Metrics Comparison', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Score')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)
axes[0].grid(alpha=0.3)
axes[0].legend(loc='lower right')

# ROC-AUC Comparison
best_model = results_df['ROC-AUC'].idxmax()
for model_name in models.keys():
    y_pred_proba = predictions[model_name]['y_pred_proba']
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc_score = results[model_name]['ROC-AUC']
    linewidth = 2.5 if model_name == best_model else 1.5
    linestyle = '-' if model_name == best_model else '--'
    axes[1].plot(fpr, tpr, label=f'{model_name} (AUC={auc_score:.3f})', linewidth=linewidth, linestyle=linestyle)

axes[1].plot([0, 1], [0, 1], 'k--', label='Random Classifier', alpha=0.5)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curves', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🏆 Best Model: {best_model} (ROC-AUC: {results[best_model]['ROC-AUC']:.4f})")

## 9. Feature Importance Analysis

Identifying which features have the greatest impact on churn predictions.

In [ ]:
# Extract feature importance from tree-based models
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Random Forest Feature Importance
rf_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': models['Random Forest'].feature_importances_
}).sort_values('importance', ascending=False).head(15)

axes[0].barh(rf_importance['feature'], rf_importance['importance'], color='#3498db', alpha=0.7)
axes[0].set_title('Random Forest - Top 15 Features', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Importance')
axes[0].invert_yaxis()

# XGBoost Feature Importance
xgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': models['XGBoost'].feature_importances_
}).sort_values('importance', ascending=False).head(15)

axes[1].barh(xgb_importance['feature'], xgb_importance['importance'], color='#e74c3c', alpha=0.7)
axes[1].set_title('XGBoost - Top 15 Features', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Importance')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\n📊 Random Forest - Top 10 Features:")
print(rf_importance.head(10).to_string(index=False))
print("\n📊 XGBoost - Top 10 Features:")
print(xgb_importance.head(10).to_string(index=False))

## 10. Churn Probability and Risk Segmentation

Generating churn probabilities and categorizing customers into risk tiers.

In [ ]:
# Generate churn probabilities using best model
best_model_obj = models[best_model]
best_churn_proba = predictions[best_model]['y_pred_proba']

# Create risk segmentation
def categorize_risk(prob):
    if prob >= 0.7:
        return 'High Risk'
    elif prob >= 0.4:
        return 'Medium Risk'
    else:
        return 'Low Risk'

risk_categories = [categorize_risk(p) for p in best_churn_proba]

# Add to test set
test_results = pd.DataFrame({
    'CustomerIndex': y_test.index,
    'Actual_Churn': y_test.values,
    'Predicted_Churn': predictions[best_model]['y_pred'],
    'Churn_Probability': best_churn_proba,
    'Risk_Category': risk_categories
})

print("🎯 Risk Segmentation Results:")
print(f"\n{test_results['Risk_Category'].value_counts()}")
print(f"\n📊 Risk Category Distribution:")
print(test_results['Risk_Category'].value_counts(normalize=True).round(3))

# Show some examples
print("\n📌 Sample High-Risk Customers:")
high_risk = test_results[test_results['Risk_Category'] == 'High Risk'].head(10)
print(high_risk[['Actual_Churn', 'Predicted_Churn', 'Churn_Probability', 'Risk_Category']].to_string())

## 11. Visualize Key Churn Drivers

Comprehensive visualizations of model performance and churn insights.

In [ ]:
# Comprehensive Visualization Dashboard
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Confusion Matrix (Best Model)
ax1 = fig.add_subplot(gs[0, 0])
cm = results[best_model]['Confusion Matrix']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1, cbar=False)
ax1.set_title(f'Confusion Matrix - {best_model}', fontweight='bold')
ax1.set_ylabel('True Label')
ax1.set_xlabel('Predicted Label')

# 2. Precision-Recall Trade-off
ax2 = fig.add_subplot(gs[0, 1])
precision_vals = [results[m]['Precision'] for m in models.keys()]
recall_vals = [results[m]['Recall'] for m in models.keys()]
ax2.scatter(recall_vals, precision_vals, s=300, alpha=0.6, c=['#3498db', '#2ecc71', '#e74c3c'])
for i, name in enumerate(models.keys()):
    ax2.annotate(name, (recall_vals[i], precision_vals[i]), fontsize=9, ha='center', fontweight='bold')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision vs Recall Trade-off', fontweight='bold')
ax2.grid(alpha=0.3)
ax2.set_xlim([0, 1])
ax2.set_ylim([0, 1])

# 3. Risk Category Distribution
ax3 = fig.add_subplot(gs[0, 2])
risk_counts = test_results['Risk_Category'].value_counts()
colors_risk = {'High Risk': '#e74c3c', 'Medium Risk': '#f39c12', 'Low Risk': '#2ecc71'}
ax3.bar(risk_counts.index, risk_counts.values, color=[colors_risk[x] for x in risk_counts.index], alpha=0.7)
ax3.set_title('Risk Category Distribution', fontweight='bold')
ax3.set_ylabel('Count')
for i, v in enumerate(risk_counts.values):
    ax3.text(i, v + 5, str(v), ha='center', fontweight='bold')

# 4. Churn Probability Distribution
ax4 = fig.add_subplot(gs[1, 0])
ax4.hist(best_churn_proba[y_test == 0], bins=30, alpha=0.6, label='Retained', color='#2ecc71')
ax4.hist(best_churn_proba[y_test == 1], bins=30, alpha=0.6, label='Churned', color='#e74c3c')
ax4.set_xlabel('Churn Probability')
ax4.set_ylabel('Count')
ax4.set_title('Predicted Churn Probability Distribution', fontweight='bold')
ax4.legend()
ax4.axvline(x=0.5, color='black', linestyle='--', linewidth=1, alpha=0.5)

# 5. Model Accuracy Comparison
ax5 = fig.add_subplot(gs[1, 1])
accuracies = [results[m]['Accuracy'] for m in models.keys()]
ax5.bar(models.keys(), accuracies, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7)
ax5.set_ylabel('Accuracy')
ax5.set_title('Model Accuracy Comparison', fontweight='bold')
ax5.set_ylim([0.7, 1])
for i, v in enumerate(accuracies):
    ax5.text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=45)

# 6. ROC-AUC Comparison
ax6 = fig.add_subplot(gs[1, 2])
auc_scores = [results[m]['ROC-AUC'] for m in models.keys()]
ax6.bar(models.keys(), auc_scores, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7)
ax6.set_ylabel('ROC-AUC Score')
ax6.set_title('ROC-AUC Comparison', fontweight='bold')
ax6.set_ylim([0.7, 1])
for i, v in enumerate(auc_scores):
    ax6.text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')
ax6.set_xticklabels(ax6.get_xticklabels(), rotation=45)

# 7. F1-Score Comparison
ax7 = fig.add_subplot(gs[2, 0])
f1_scores = [results[m]['F1-Score'] for m in models.keys()]
ax7.bar(models.keys(), f1_scores, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7)
ax7.set_ylabel('F1-Score')
ax7.set_title('F1-Score Comparison', fontweight='bold')
ax7.set_ylim([0.4, 0.8])
for i, v in enumerate(f1_scores):
    ax7.text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')
ax7.set_xticklabels(ax7.get_xticklabels(), rotation=45)

# 8. Churn Probability by Risk Category
ax8 = fig.add_subplot(gs[2, 1])
risk_order = ['Low Risk', 'Medium Risk', 'High Risk']
churn_by_risk = [best_churn_proba[test_results['Risk_Category'] == cat].mean() for cat in risk_order]
ax8.bar(risk_order, churn_by_risk, color=['#2ecc71', '#f39c12', '#e74c3c'], alpha=0.7)
ax8.set_ylabel('Average Churn Probability')
ax8.set_title('Average Churn Probability by Risk Category', fontweight='bold')
for i, v in enumerate(churn_by_risk):
    ax8.text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# 9. Actual Churn Rate by Risk Category
ax9 = fig.add_subplot(gs[2, 2])
actual_churn_by_risk = [y_test[test_results['Risk_Category'] == cat].mean() for cat in risk_order]
ax9.bar(risk_order, actual_churn_by_risk, color=['#2ecc71', '#f39c12', '#e74c3c'], alpha=0.7)
ax9.set_ylabel('Actual Churn Rate')
ax9.set_title('Actual Churn Rate by Risk Category', fontweight='bold')
for i, v in enumerate(actual_churn_by_risk):
    ax9.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

fig.suptitle('🔮 Churn Prediction - Comprehensive Dashboard', fontsize=16, fontweight='bold', y=0.995)
plt.show()

## 12. Generate Business Insights Report

Key findings and actionable recommendations for decision-makers.

In [ ]:
print("\n" + "="*80)
print("💼 BUSINESS INSIGHTS REPORT - CHURN PREDICTION SYSTEM")
print("="*80)

print(f"\n📊 EXECUTIVE SUMMARY")
print(f"-" * 80)
print(f"Model: {best_model}")
print(f"Overall Churn Rate: {churn_rate:.1f}%")
print(f"Model Accuracy: {results[best_model]['Accuracy']:.1%}")
print(f"Model ROC-AUC: {results[best_model]['ROC-AUC']:.1%}")

print(f"\n🎯 CUSTOMER SEGMENTATION")
print(f"-" * 80)
for risk_cat in ['High Risk', 'Medium Risk', 'Low Risk']:
    count = len(test_results[test_results['Risk_Category'] == risk_cat])
    pct = count / len(test_results) * 100
    print(f"{risk_cat:15} : {count:4} customers ({pct:5.1f}%)")

print(f"\n🔍 KEY CHURN DRIVERS (XGBoost)")
print(f"-" * 80)
top_features = xgb_importance.head(5)
for idx, row in top_features.iterrows():
    print(f"  {row['feature']:25} → Importance: {row['importance']:.4f}")

print(f"\n💡 ACTIONABLE RECOMMENDATIONS")
print(f"-" * 80)
print(f"""
1. 🎯 TARGET HIGH-RISK CUSTOMERS
   - Identified {len(test_results[test_results['Risk_Category'] == 'High Risk'])} high-risk customers
   - Implement retention campaigns focusing on top churn drivers
   - Offer personalized retention incentives

2. 📞 PROACTIVE OUTREACH
   - Contact customers before churn occurs
   - Use predicted probabilities to prioritize engagement
   - Focus resources on medium-risk customers (highest ROI opportunity)

3. 🔧 PRODUCT IMPROVEMENTS
   - Analyze features driving churn (tenure, charges, contract type)
   - Improve onboarding for new customers (tenure < 6 months)
   - Review pricing strategy for high-charge customers

4. 📈 MONITORING & IMPROVEMENT
   - Track model performance over time
   - Retrain quarterly with new customer data
   - Monitor false positives to avoid over-retention spending

5. 💰 FINANCIAL IMPACT
   - Prioritize retention of high-value customers
   - Allocate retention budget to high-risk segment
   - Expected ROI: Cost of retention << Cost of acquisition
""")

print(f"\n✅ MODEL RELIABILITY METRICS")
print(f"-" * 80)
print(f"Precision (Identifying actual churners): {results[best_model]['Precision']:.1%}")
print(f"  → Of customers we predict will churn, {results[best_model]['Precision']:.1%} actually do")
print(f"\nRecall (Catching all churners): {results[best_model]['Recall']:.1%}")
print(f"  → We successfully identify {results[best_model]['Recall']:.1%} of all customers who churn")
print(f"\nF1-Score (Balance): {results[best_model]['F1-Score']:.3f}")
print(f"  → Good balance between precision and recall")

print("\n" + "="*80)